In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd
from selenium.common.exceptions import NoSuchElementException, TimeoutException

def redbus(links):
    # Initialize WebDriver
    driver = webdriver.Chrome()
    wait = WebDriverWait(driver, 30)  # Timeout for element waits

    # Navigate to the URL
    driver.get(links)

    # Define a list to store all the bus details
    all_data = []

    def scrape_route_details(route_name, route_link):
        # Open the route link in a new tab
        driver.execute_script(f"window.open('{route_link}');")
        driver.switch_to.window(driver.window_handles[1])
        try:
            view_buses_button = WebDriverWait(driver, 40).until(EC.element_to_be_clickable((By.CLASS_NAME, 'button')))
            view_buses_button.click()
            time.sleep(5)
        except NoSuchElementException:
            pass
        try:
            # Scroll to the bottom to load all buses
            last_height = driver.execute_script("return document.body.scrollHeight")
            while True:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)  # Wait to load the page
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            # Extract bus details from the page
          
            try:
                    bus_name = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.travels.lh-24.f-bold.d-color'))).text
                    bus_type = driver.find_element(By.CSS_SELECTOR, 'div.bus-type.f-12.m-top-16.l-color.evBus').text
                    departing_time = driver.find_element(By.CSS_SELECTOR, 'div.dp-time.f-19.d-color.f-bold').text
                    duration = driver.find_element(By.CSS_SELECTOR, 'div.dur.l-color.lh-24').text
                    reaching_time = driver.find_element(By.CSS_SELECTOR, 'div.bp-time.f-19.d-color.disp-Inline').text
                    star_rating = float(driver.find_element(By.CSS_SELECTOR, 'div.rating-sec.lh-24').text)
                    price = float(driver.find_element(By.CSS_SELECTOR, 'span.f-19.f-bold').text.replace("₹", "").replace(",", ""))
                    
                    try:
                        seats_available = driver.find_element(By.CSS_SELECTOR, "div.seat-left.m-top-16").text
                    except NoSuchElementException:
                        seats_available = driver.find_element(By.CSS_SELECTOR, "div.seat-left.m-top-30").text.split()[0]

                # Append extracted data to the list including route details
                    all_data.append({
                        'route_name': route_name,
                        'route_link': route_link,
                        'bus_name': bus_name,
                        'bus_type': bus_type,
                        'departing_time': departing_time,
                        'duration': duration,
                        'reaching_time': reaching_time,
                        'star_rating': star_rating,
                        'price': price,
                        'seats_available': seats_available
                })
            except NoSuchElementException as e:
                print(f"An error occurred while extracting data from a bus entry in {route_link}: {e}")
        except NoSuchElementException as e:
            print(f"An error occurred while extracting data from {route_link}: {e}")
        finally:
            # Close the new tab and switch back to the main tab
            driver.close()
            driver.switch_to.window(driver.window_handles[0])

    def scrape_page():
        # Locate elements (routes container)
        try:
            routescontainer = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//a[@class='route']")))

            # Loop through each route to extract details
            for route in routescontainer:
                try:
                    # Extract route name and link
                    route_name = route.text
                    route_link = route.get_attribute('href')

                    # Extract bus details from each route link
                    scrape_route_details(route_name, route_link)

                except Exception as e:
                    print(f"An error occurred while processing route: {e}")
                    continue
        except TimeoutException:
            print("Failed to load routes container.")
        # Wait for the page to load completely
        time.sleep(5)

    # Main loop for scraping multiple pages
    page_number = 1
    while page_number <= 5:
        scrape_page()
        try:
            if page_number >= 5:  # Stop after the 5th page
                break
            # Locate the pagination container
            pagination_container = wait.until(EC.presence_of_element_located(
                (By.XPATH, '//div[contains(@class, "DC_117_paginationTable")]')
            ))

            # Locate the next page button within the container
            next_page_button = pagination_container.find_element(
                By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
            )

            # Ensure the next page button is in view
            actions = ActionChains(driver)
            actions.move_to_element(next_page_button).perform()
            time.sleep(1)  # Wait for a bit after scrolling

            # Click the next page button
            next_page_button.click()

            # Wait for the page number to update to the next page
            wait.until(EC.text_to_be_present_in_element(
                (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))

            # Wait for a short duration to ensure the next page loads completely
            time.sleep(3)

            page_number += 1

        except (NoSuchElementException, TimeoutException) as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break

    # Convert the list of dictionaries to a DataFrame
    df_T = pd.DataFrame(all_data)

    # Save the DataFrame to a CSV file
    df_T.to_csv('usrtc.csv', index=False)

    # Close the WebDriver
    driver.quit()

    return df_T  # Return the DataFrame instead of None

# Example usage
# df = redbus('your_starting_url_here')
# print(df)


In [4]:
links = "https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile"
bus_full_details = redbus(links)


An error occurred while navigating to page 3: Message: no such element: Unable to locate element: {"method":"xpath","selector":".//div[contains(@class, "DC_117_pageTabs") and text()="3"]"}
  (Session info: chrome=127.0.6533.122); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6F92B9642+30946]
	(No symbol) [0x00007FF6F926E3D9]
	(No symbol) [0x00007FF6F9166FDA]
	(No symbol) [0x00007FF6F91B822C]
	(No symbol) [0x00007FF6F91B850C]
	(No symbol) [0x00007FF6F91ABCDC]
	(No symbol) [0x00007FF6F91DCAAF]
	(No symbol) [0x00007FF6F91ABBB6]
	(No symbol) [0x00007FF6F91DCC80]
	(No symbol) [0x00007FF6F91FB041]
	(No symbol) [0x00007FF6F91DC813]
	(No symbol) [0x00007FF6F91AA6E5]
	(No symbol) [0x00007FF6F91AB021]
	GetHandleVerifier [0x00007FF6F93EF84D+1301229]
	GetHandleVerifier [0x00007FF6F93FBDC7+1351783]
	GetHandleVerifier [0x00007FF6F93F2A13+1313971]
	GetHandleVerif